In [1]:
import json
import gurobipy as gp
from gurobipy import GRB
import numpy as np
import matplotlib.pyplot as plt

In [50]:
status_codes = {
    1: 'LOADED',
    2: 'OPTIMAL',
    3: 'INFEASIBLE',
    4: 'INF_OR_UNBD',
    5: 'UNBOUNDED',
    6: 'CUTOFF',
    7: 'ITERATION_LIMIT',
    8: 'NODE_LIMIT',
    9: 'TIME_LIMIT',
    10: 'SOLUTION_LIMIT',
    11: 'INTERRUPTED',
    12: 'NUMERIC',
    13: 'SUBOPTIMAL',
    14: 'INPROGRESS',
    15: 'USER_OBJ_LIMIT'
}

In [51]:
rng = np.random.default_rng(8)

In [52]:
A_3_1 = np.array([
    [1, 0, 0, 0],
    [0, -1, 0, 0]
])

A_3_2 = np.array([
    [1, 2, 0, 0],
    [0, 1, -2, 0]
])

A_3_3 = np.array([
    [1, 3, 3, 0],
    [0, -1, 3, -3]
])

In [53]:
k1, k2 = 2, 1

delta = 10

gamma = np.array([
    0, 0, 0, 0#0, 4, 2, -5
])

lam = k1 / k2
y_true = np.array([
    1,
    lam,
    lam**2 + lam,
    lam**3 + 3*(lam**2) + lam
])

y_true += gamma

y_lb = y_true - (delta / 2)
y_ub = y_true + (delta / 2)

y_lb[0] = 1
y_ub[0] = 1

In [54]:
options = {}
options['OutputFlag'] = 0
time_limit = 300

# environment context
with gp.Env(params=options) as env:

    # model context
    with gp.Model('test-SDP', env=env) as model:

        # model settings
        model.Params.TimeLimit = time_limit
        K = 100

        # variables
        y = model.addMVar(shape=4, vtype=GRB.CONTINUOUS, name="y", lb=0)
        k = model.addMVar(shape=2, vtype=GRB.CONTINUOUS, name="k", lb=0, ub=K)
        M_0 = gp.MVar.fromlist([
            [y[0].item(), y[1].item()],
            [y[1].item(), y[2].item()]
        ])
        M_1 = gp.MVar.fromlist([
            [y[1].item(), y[2].item()],
            [y[2].item(), y[3].item()]
        ])

        # constraints

        # moment bounds
        model.addConstr(y <= y_ub, name="y_UB")
        model.addConstr(y >= y_lb, name="y_LB")

        # moment equations
        model.addConstr(k.T @ A_3_1 @ y == 0, name="ME_1")
        #model.addConstr(k.T @ A_3_2 @ y == 0, name="ME_2")
        #model.addConstr(k.T @ A_3_3 @ y == 0, name="ME_3")

        # base
        model.addConstr(y[0] == 1, name="y0_base")

        # fixed parameter
        model.addConstr(k[1] == k2, name="k2_fixed")

        # optimize
        solution = {}
        model.setObjective(0, GRB.MINIMIZE)
        model.optimize()
        status = status_codes[model.status]
        print(f"Status {status}")

        # found feasible point
        while status == "OPTIMAL":

            # get moment matrix values
            M_0_val = M_0.X
            M_1_val = M_1.X

            # check semidefinite
            evals_0, evecs_0 = np.linalg.eigh(M_0_val)
            evals_1, evecs_1 = np.linalg.eigh(M_1_val)

            print(evals_0)
            print(evals_1)

            # positive eigenvalues
            if (evals_0 >= 0).all() and (evals_1 >= 0).all():
                print("SDP feasible")
                break

            # negative eigenvalue
            else:

                if evals_0[0] < 0:

                    # get eigenvector
                    v = evecs_0[0]

                    # add cutting plane
                    model.addConstr(v.T @ M_0 @ v >= 0, name="Cut_0")

                    print("M_0 cut")

                if evals_1[0] < 0:

                    # get eigenvector
                    v = evecs_1[0]

                    # add cutting plane
                    model.addConstr(v.T @ M_1 @ v >= 0, name="Cut_1")

                    print("M_1 cut")

                # resolve
                solution = {}
                model.setObjective(0, GRB.MINIMIZE)
                model.optimize()
                status = status_codes[model.status]
                print(f"Status {status}")

Status OPTIMAL
[-2.02282456  4.02282456]
[ 2.95164187 17.0711827 ]
M_0 cut
Status OPTIMAL
[ 0.36852324 11.63147676]
[-3.35379048 22.94483863]
M_1 cut
Status OPTIMAL
[0.09381891 7.43980674]
[ 0.78717896 28.62852613]
SDP feasible


## Notes

Moment equations very good for such a simple model, strong enough that valid moments are feasible and also satisify SDP (so end with no cuts needed) or invalid moments infeasible (so end with no cuts)

Reducing to 1 moment equation weakens constraints so we can test cuts:

Valid moments with very weak bounds (e.g. delta = 10), feasible relaxation point infeasible under SDP so add cut, repeat a few times until feasible (done)

Invalid moments (e.g. gamma = [0, 4, 2, -5]), feasible relaxation point infeasible under SDP so add cut, relaxation is then infeasible (done)

Overall very good performance

### TODO

Clean up code, use functions (model creation, adding cuts, etc) to make loop easier to work with

## Function version

In [55]:
def base_model(model):

    # model settings
    model.Params.TimeLimit = time_limit
    K = 100

    # variables
    y = model.addMVar(shape=4, vtype=GRB.CONTINUOUS, name="y", lb=0)
    k = model.addMVar(shape=2, vtype=GRB.CONTINUOUS, name="k", lb=0, ub=K)
    M_0 = gp.MVar.fromlist([
        [y[0].item(), y[1].item()],
        [y[1].item(), y[2].item()]
    ])
    M_1 = gp.MVar.fromlist([
        [y[1].item(), y[2].item()],
        [y[2].item(), y[3].item()]
    ])

    # constraints

    # moment bounds
    model.addConstr(y <= y_ub, name="y_UB")
    model.addConstr(y >= y_lb, name="y_LB")

    # moment equations
    model.addConstr(k.T @ A_3_1 @ y == 0, name="ME_1")
    #model.addConstr(k.T @ A_3_2 @ y == 0, name="ME_2")
    #model.addConstr(k.T @ A_3_3 @ y == 0, name="ME_3")

    # base
    model.addConstr(y[0] == 1, name="y0_base")

    # fixed parameter
    model.addConstr(k[1] == k2, name="k2_fixed")

    # variable dict
    variables = {
        'y': y,
        'k': k,
        'M_0': M_0,
        'M_1': M_1
    }

    return model, variables

In [56]:
def optimize(model):

    # optimize
    model.setObjective(0, GRB.MINIMIZE)
    model.optimize()
    status = status_codes[model.status]

    return model, status

In [57]:
def semidefinite_cut(model, variables, print_evals=False):
    '''
    Check semidefinite feasibility of optimal moment values
    
    Feasible: stop
    Infeasible: add cutting plane

    Returns:
        model
        (bool) semidefinite feasibility
    '''

    # get moment matrix values
    M_0_val = variables['M_0'].X
    M_1_val = variables['M_1'].X

    # check semidefinite
    evals_0, evecs_0 = np.linalg.eigh(M_0_val)
    evals_1, evecs_1 = np.linalg.eigh(M_1_val)

    if print_evals:
        print("Moment matices eigenvalues:")
        print(evals_0)
        print(evals_1)

    # positive eigenvalues
    if (evals_0 >= 0).all() and (evals_1 >= 0).all():

        print("SDP feasible\n")
    
        return model, True

    # negative eigenvalue
    else:

        print("SDP infeasible")

        if evals_0[0] < 0:

            # get eigenvector
            v = evecs_0[0]

            # add cutting plane
            model.addConstr(v.T @ variables['M_0'] @ v >= 0, name="Cut_0")

            print("M_0 cut added\n")

        if evals_1[0] < 0:

            # get eigenvector
            v = evecs_1[0]

            # add cutting plane
            model.addConstr(v.T @ variables['M_1'] @ v >= 0, name="Cut_1")

            print("M_1 cut added\n")

    return model, False

In [58]:
options = {}
options['OutputFlag'] = 0
time_limit = 300

# environment context
with gp.Env(params=options) as env:

    # model context
    with gp.Model('test-SDP', env=env) as model:

        # construct base model (no semidefinite constraints)
        model, variables = base_model(model)

        # check feasibility
        model, status = optimize(model)

        # while feasible
        while status == "OPTIMAL":

            print("NLP feasible")

            # check semidefinite feasibility
            model, semidefinite_feas = semidefinite_cut(model, variables, print_evals=True)

            # semidefinite feasible
            if semidefinite_feas:
                break

            # semidefinite infeasible
            else:

                # check feasibility
                model, status = optimize(model)

        # if infeasible
        if status == "INFEASIBLE":
            print("SDP infeasible")


NLP feasible
Moment matices eigenvalues:
[-2.02282456  4.02282456]
[ 2.95164187 17.0711827 ]
SDP infeasible
M_0 cut added

NLP feasible
Moment matices eigenvalues:
[ 0.36852324 11.63147676]
[-3.35379048 22.94483863]
SDP infeasible
M_1 cut added

NLP feasible
Moment matices eigenvalues:
[0.09381891 7.43980674]
[ 0.78717896 28.62852613]
SDP feasible



## Symbolic A matrices

In [59]:
import sympy as sp

In [60]:
x = sp.Symbol('x')

# reaction propensity polynomials
props = [
    1,
    x
]

# reaction vectors
vrs = [
    1,
    -1
]

alpha = 1
Nd = 4
R = 2

A = np.zeros((R, Nd))

for r, prop in enumerate(props):

    # expand b(x) * ((x + v_r)**alpha - x**alpha)
    poly = sp.Poly(prop * ((x + vrs[r])**alpha - x**alpha), x)

    # collect coefficients
    coeffs = poly.all_coeffs()

    # reverse
    coeffs.reverse()

    # put coefficients in matrix
    for l in range(Nd):

        try:
            coeff = coeffs[l]
        except IndexError:
            coeff = 0

        A[r, l] = coeff

print(A)

[[ 1.  0.  0.  0.]
 [ 0. -1.  0.  0.]]


In [61]:
def compute_A(alpha, Nd=4):

    x = sp.Symbol('x')
    
    # reaction settings
    R = 2

    # reaction propensity polynomials
    props = [
        1,
        x
    ]

    # reaction vectors
    vrs = [
        1,
        -1
    ]

    # setup matrix
    A = np.zeros((R, Nd))

    for r, prop in enumerate(props):

        # expand b(x) * ((x + v_r)**alpha - x**alpha)
        poly = sp.Poly(prop * ((x + vrs[r])**alpha - x**alpha), x)

        # collect coefficients
        coeffs = poly.all_coeffs()

        # reverse
        coeffs.reverse()

        # put coefficients in matrix
        for l in range(Nd):

            try:
                coeff = coeffs[l]
            except IndexError:
                coeff = 0

            A[r, l] = coeff

    return A

In [62]:
compute_A(2)

array([[ 1.,  2.,  0.,  0.],
       [ 0.,  1., -2.,  0.]])

## Bootstrap estimates

In [63]:
A_3_1 = compute_A(1)
A_3_2 = compute_A(2)
A_3_3 = compute_A(3)

In [64]:
def base_model(model, y_bounds):

    # model settings
    model.Params.TimeLimit = time_limit
    K = 100

    # variables
    y = model.addMVar(shape=4, vtype=GRB.CONTINUOUS, name="y", lb=0)
    k = model.addMVar(shape=2, vtype=GRB.CONTINUOUS, name="k", lb=0, ub=K)
    M_0 = gp.MVar.fromlist([
        [y[0].item(), y[1].item()],
        [y[1].item(), y[2].item()]
    ])
    M_1 = gp.MVar.fromlist([
        [y[1].item(), y[2].item()],
        [y[2].item(), y[3].item()]
    ])

    # constraints
    y_lb = y_bounds[0, :]
    y_ub = y_bounds[1, :]

    # moment bounds
    model.addConstr(y <= y_ub, name="y_UB")
    model.addConstr(y >= y_lb, name="y_LB")

    # moment equations
    model.addConstr(k.T @ A_3_1 @ y == 0, name="ME_1")
    #model.addConstr(k.T @ A_3_2 @ y == 0, name="ME_2")
    #model.addConstr(k.T @ A_3_3 @ y == 0, name="ME_3")

    # base
    model.addConstr(y[0] == 1, name="y0_base")

    # fixed parameter
    model.addConstr(k[1] == k2, name="k2_fixed")

    # variable dict
    variables = {
        'y': y,
        'k': k,
        'M_0': M_0,
        'M_1': M_1
    }

    return model, variables

In [88]:
# settings
k1, k2 = 3, 1
n = 1000
N = 1000
d = 3

# sample
sample = rng.poisson(k1 / k2, size=n)

# bootstrap
bootstrap = rng.choice(sample, size=(N, n))

# estimate
y_bounds = np.zeros((2, d + 1))
for l in range(d + 1):
    
    moment_estimates = np.mean(bootstrap**l, axis=1)

    moment_interval = np.quantile(moment_estimates, [0.025, 0.975])

    y_bounds[:, l] = moment_interval

In [89]:
# gamma bias
gamma = np.array([0, -2, 2, 5])

#y_bounds += gamma

In [90]:
lam = k1 / k2
y_true = np.array([
    1,
    lam,
    lam**2 + lam,
    lam**3 + 3*(lam**2) + lam
])

for l in range(d + 1):
    if y_bounds[0, l] <= y_true[l] and y_true[l] <= y_bounds[1, l]:
        print(f"True value {y_true[l]} in CI ({y_bounds[0, l]}, {y_bounds[1, l]})")
    else:
        print(f"True value {y_true[l]} not in CI ({y_bounds[0, l]}, {y_bounds[1, l]})")

True value 1.0 in CI (1.0, 1.0)
True value 3.0 in CI (2.927975, 3.145025)
True value 12.0 in CI (11.55095, 13.255725)
True value 57.0 in CI (53.717475, 66.4473)


In [91]:
options = {}
options['OutputFlag'] = 0
time_limit = 300

# environment context
with gp.Env(params=options) as env:

    # model context
    with gp.Model('test-SDP', env=env) as model:

        # construct base model (no semidefinite constraints)
        model, variables = base_model(model, y_bounds)

        # check feasibility
        model, status = optimize(model)

        # while feasible
        while status == "OPTIMAL":

            print("NLP feasible")

            # check semidefinite feasibility
            model, semidefinite_feas = semidefinite_cut(model, variables, print_evals=True)

            # semidefinite feasible
            if semidefinite_feas:
                break

            # semidefinite infeasible
            else:

                # check feasibility
                model, status = optimize(model)

        # if infeasible
        if status == "INFEASIBLE":
            print("SDP infeasible")

NLP feasible
Moment matices eigenvalues:
[ 0.24054299 12.31040701]
[ 0.42710529 56.22120028]
SDP feasible



## Capture efficiency

In [136]:
from scipy.special import stirling2

In [200]:
# sample
beta = rng.beta(1, 2, size=1000)

# compute moments
y_beta = np.zeros(d + 1)
for l in range(d + 1):
    y_beta[l] = np.mean(beta**l)

print(y_beta)

[1.         0.34985842 0.17977954 0.11072379]


In [201]:
# matrix
B = np.array([
    [1, 0, 0, 0],
    [0, y_beta[1], 0, 0],
    [0, -y_beta[2] + y_beta[1], y_beta[2], 0],
    [0, 2*y_beta[3] - 3*y_beta[2] + y_beta[1], -3*y_beta[3] + 3*y_beta[2], y_beta[3]]
])

print(B)

[[1.         0.         0.         0.        ]
 [0.         0.34985842 0.         0.        ]
 [0.         0.17007888 0.17977954 0.        ]
 [0.         0.03196737 0.20716727 0.11072379]]


In [202]:
def compute_B(beta, d):

    # compute beta moments
    y_beta = np.zeros(d + 1)
    for l in range(d + 1):
        y_beta[l] = np.mean(beta**l)

    n, p = sp.symbols('n p')

    def falling_factorial(n, k):
        val = 1
        for i in range(k):
            val *= (n - i)
        return val

    def binomial_moment(l):
        val = 0
        for k in range(l + 1):
            val += falling_factorial(n, k) * stirling2(l, k) * p**k
        return val

    B = np.zeros((d + 1, d + 1))

    for l in range(d + 1):

        moment = binomial_moment(l)

        poly = sp.Poly(moment, n, p)

        for (j, k), coeff in zip(poly.monoms(), poly.coeffs()):

            B[l, j] += coeff * y_beta[k]

    return B

In [203]:
compute_B(beta, 3)

array([[1.        , 0.        , 0.        , 0.        ],
       [0.        , 0.34985842, 0.        , 0.        ],
       [0.        , 0.17007888, 0.17977954, 0.        ],
       [0.        , 0.03196737, 0.20716727, 0.11072379]])

## Setup with capture efficiency

In [204]:
def compute_A(alpha, Nd=4):

    x = sp.Symbol('x')
    
    # reaction settings
    R = 2

    # reaction propensity polynomials
    props = [
        1,
        x
    ]

    # reaction vectors
    vrs = [
        1,
        -1
    ]

    # setup matrix
    A = np.zeros((R, Nd))

    for r, prop in enumerate(props):

        # expand b(x) * ((x + v_r)**alpha - x**alpha)
        poly = sp.Poly(prop * ((x + vrs[r])**alpha - x**alpha), x)

        # collect coefficients
        coeffs = poly.all_coeffs()

        # reverse
        coeffs.reverse()

        # put coefficients in matrix
        for l in range(Nd):

            try:
                coeff = coeffs[l]
            except IndexError:
                coeff = 0

            A[r, l] = coeff

    return A

In [205]:
A_3_1 = compute_A(1)
A_3_2 = compute_A(2)
A_3_3 = compute_A(3)

In [206]:
def optimize(model):

    # optimize
    model.setObjective(0, GRB.MINIMIZE)
    model.optimize()
    status = status_codes[model.status]

    return model, status

In [207]:
def semidefinite_cut(model, variables, print_evals=False):
    '''
    Check semidefinite feasibility of optimal moment values
    
    Feasible: stop
    Infeasible: add cutting plane

    Returns:
        model
        (bool) semidefinite feasibility
    '''

    # get moment matrix values
    M_0_val = variables['M_0'].X
    M_1_val = variables['M_1'].X

    # check semidefinite
    evals_0, evecs_0 = np.linalg.eigh(M_0_val)
    evals_1, evecs_1 = np.linalg.eigh(M_1_val)

    if print_evals:
        print("Moment matices eigenvalues:")
        print(evals_0)
        print(evals_1)

    # positive eigenvalues
    if (evals_0 >= 0).all() and (evals_1 >= 0).all():

        print("SDP feasible\n")
    
        return model, True

    # negative eigenvalue
    else:

        print("SDP infeasible")

        if evals_0[0] < 0:

            # get eigenvector
            v = evecs_0[0]

            # add cutting plane
            model.addConstr(v.T @ variables['M_0'] @ v >= 0, name="Cut_0")

            print("M_0 cut added\n")

        if evals_1[0] < 0:

            # get eigenvector
            v = evecs_1[0]

            # add cutting plane
            model.addConstr(v.T @ variables['M_1'] @ v >= 0, name="Cut_1")

            print("M_1 cut added\n")

    return model, False

In [208]:
def base_model(model, y_OB_bounds, B):

    # model settings
    model.Params.TimeLimit = time_limit
    K = 100

    # variables
    y = model.addMVar(shape=4, vtype=GRB.CONTINUOUS, name="y", lb=0)
    k = model.addMVar(shape=2, vtype=GRB.CONTINUOUS, name="k", lb=0, ub=K)
    M_0 = gp.MVar.fromlist([
        [y[0].item(), y[1].item()],
        [y[1].item(), y[2].item()]
    ])
    M_1 = gp.MVar.fromlist([
        [y[1].item(), y[2].item()],
        [y[2].item(), y[3].item()]
    ])

    # constraints
    y_lb = y_OB_bounds[0, :]
    y_ub = y_OB_bounds[1, :]

    # moment bounds
    model.addConstr(B @ y <= y_ub, name="y_UB")
    model.addConstr(B @ y >= y_lb, name="y_LB")

    # moment equations
    model.addConstr(k.T @ A_3_1 @ y == 0, name="ME_1")
    #model.addConstr(k.T @ A_3_2 @ y == 0, name="ME_2")
    #model.addConstr(k.T @ A_3_3 @ y == 0, name="ME_3")

    # base
    model.addConstr(y[0] == 1, name="y0_base")

    # fixed parameter
    model.addConstr(k[1] == k2, name="k2_fixed")

    # variable dict
    variables = {
        'y': y,
        'k': k,
        'M_0': M_0,
        'M_1': M_1
    }

    return model, variables

In [209]:
def compute_B(beta, d):

    # compute beta moments
    y_beta = np.zeros(d + 1)
    for l in range(d + 1):
        y_beta[l] = np.mean(beta**l)

    n, p = sp.symbols('n p')

    def falling_factorial(n, k):
        val = 1
        for i in range(k):
            val *= (n - i)
        return val

    def binomial_moment(l):
        val = 0
        for k in range(l + 1):
            val += falling_factorial(n, k) * stirling2(l, k) * p**k
        return val

    B = np.zeros((d + 1, d + 1))

    for l in range(d + 1):

        moment = binomial_moment(l)

        poly = sp.Poly(moment, n, p)

        for (j, k), coeff in zip(poly.monoms(), poly.coeffs()):

            B[l, j] += coeff * y_beta[k]

    return B

In [210]:
# settings
k1, k2 = 3, 1
b = 2
n = 1000
N = 1000
d = 3

# sample
sample = rng.poisson(k1 / k2, size=n)

# downsample
beta = rng.beta(1, b, size=1000)
downsample = rng.binomial(sample, beta)

# compute scaling matrix
B = compute_B(beta, d)

# bootstrap
bootstrap = rng.choice(downsample, size=(N, n))

# estimate
y_bounds = np.zeros((2, d + 1))
for l in range(d + 1):
    
    moment_estimates = np.mean(bootstrap**l, axis=1)

    moment_interval = np.quantile(moment_estimates, [0.025, 0.975])

    y_bounds[:, l] = moment_interval

In [212]:
# gamma bias
gamma = np.array([0, -1, 2, 1])

y_bounds += gamma

In [213]:
options = {}
options['OutputFlag'] = 0
time_limit = 300

# environment context
with gp.Env(params=options) as env:

    # model context
    with gp.Model('test-SDP', env=env) as model:

        # construct base model (no semidefinite constraints)
        model, variables = base_model(model, y_bounds, B)

        # check feasibility
        model, status = optimize(model)

        # while feasible
        while status == "OPTIMAL":

            print("NLP feasible")

            # check semidefinite feasibility
            model, semidefinite_feas = semidefinite_cut(model, variables, print_evals=True)

            # semidefinite feasible
            if semidefinite_feas:
                break

            # semidefinite infeasible
            else:

                # check feasibility
                model, status = optimize(model)

        # if infeasible
        if status == "INFEASIBLE":
            print("SDP infeasible")

NLP feasible
Moment matices eigenvalues:
[ 0.99870587 25.48207028]
[-11.04808834  58.0138248 ]
SDP infeasible
M_1 cut added

SDP infeasible
